In [9]:
from functools import partial
from tqdm import tqdm

import numpy as np
import jax
from sbi_lens.simulator.utils import gaussian_log_likelihood
from sbi_lens.simulator.LogNormal_field import lensingLogNormal
from sbi_lens.config import config_lsst_y_10


from sbi_lens.metrics.c2st import c2st


######## CONFIG LSST Y 10 ########
print('######## CONFIG LSST Y 10 ########')
dim              = 6

N                = config_lsst_y_10.N
map_size         = config_lsst_y_10.map_size
sigma_e          = config_lsst_y_10.sigma_e
gals_per_arcmin2 = config_lsst_y_10.gals_per_arcmin2
nbins            = config_lsst_y_10.nbins
a                = config_lsst_y_10.a
b                = config_lsst_y_10.b
z0               = config_lsst_y_10.z0

truth            = config_lsst_y_10.truth

params_name      = config_lsst_y_10.params_name_latex


######## DATASET ########
print('######## DATASET ########')

dataset = np.load('./LOADED&COMPRESSED_year_10_with_noise_score_density_proposal.npz', allow_pickle=True)['arr_0']

m_data = np.load('./data/m_data__256N_10ms_27gpa_0.26se.npy')

######## LOSSES & UPDATE FUN ########
print('######## LOSSES & UPDATE FUN ########')

log_likelihood = partial(
    gaussian_log_likelihood, 
    mass_map = np.array(m_data),
    map_size = map_size,
    sigma_e = sigma_e,
    a = a,
    b = b,
    z0 = z0,
    gals_per_arcmin2 = gals_per_arcmin2
)

prob_anlytic_likelihood_fn = jax.jit(jax.vmap(log_likelihood))

probs = []
for i in tqdm(range(100_000 // 100)):
  probs.append(prob_anlytic_likelihood_fn(dataset.item()['theta'][i*100:(i+1)*100]))
probs = np.concatenate(probs)

######## CONFIG LSST Y 10 ########
######## DATASET ########
######## LOSSES & UPDATE FUN ########


  0%|          | 0/1000 [00:00<?, ?it/s]/linkhome/rech/genkqu01/ufa23yn/.local/lib/python3.10/site-packages/jax_cosmo/scipy/interpolate.py:35: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  s = np.sign(np.clip(x, xp[1], xp[-2]) - xi).astype(np.int64)
/linkhome/rech/genkqu01/ufa23yn/.local/lib/python3.10/site-packages/jax_cosmo/scipy/interpolate.py:36: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  a = (fp[ind + np.copysign(1, s).astype(np.int64)] -

In [10]:
probs

array([31990.424, 31991.385, 31990.23 , ..., 31989.08 , 31989.406,
       31988.945], dtype=float32)

In [5]:
np.save('probs_ps_likelihood.npy', probs)